# Motor Control Testing

Test notebook for the **DigitalTwinMotorDriver** and **Motor** device class.

Uses the new telescope-mcp architecture - no hardware required!

---

## Architecture Overview

```
Motor (device)          ← High-level async API
    ↓
MotorInstance           ← Runtime instance (position, status)
    ↓
MotorDriver             ← Factory for instances (DigitalTwinMotorDriver)
```

## Motor Configuration

| Axis | Range | Home | Steps/Degree |
|------|-------|------|--------------|
| ALT | 0° (zenith) → ~90° (horizon) | 0 (zenith) | ~19,167 |
| AZ | ±110,000 steps (~±50°) | 0 (center) | ~2,222 |

---
## 1. Configuration

Toggle between **Digital Twin** (simulation) and **Hardware** (real motors) mode.

In [1]:
# ============================================
# CONFIGURATION - Choose hardware or simulation
# ============================================

# Set to True for real hardware, False for digital twin simulation
#USE_HARDWARE = False
USE_HARDWARE = True

# Hardware settings (only used when USE_HARDWARE=True)
MOTOR_BAUD_RATE = 115200

# Known Teensy hardware ID (from serial port detection)
# Format: "USB VID:PID=16C0:0483 SER=<serial_number> LOCATION=<bus-port>"
# Run the port detection cell to find your Teensy's hwid, then paste it here
TEENSY_HWID = None  # Set to string like "USB VID:PID=16C0:0483 SER=12345678 LOCATION=1-2"

print(f"✓ Mode: {'HARDWARE' if USE_HARDWARE else 'DIGITAL TWIN'}")
if TEENSY_HWID:
    print(f"✓ Teensy hwid: {TEENSY_HWID}")

✓ Mode: HARDWARE


In [2]:
# ============================================
# DETECT SERIAL PORTS
# Shows available ports and identifies Teensy
# Only runs detection in hardware mode
# ============================================

import serial.tools.list_ports

teensy_port = None

if USE_HARDWARE:
    print("Scanning serial ports...\n")
    ports = list(serial.tools.list_ports.comports())

    # Look for Teensy (Teensyduino shows as "USB Serial" or contains "Teensy")
    for port in ports:
        desc = (port.description or "").lower()
        mfr = (port.manufacturer or "").lower()
        
        # Teensy typically shows as "USB Serial" with Teensyduino or PJRC
        is_teensy = (
            "teensy" in desc or 
            "teensy" in mfr or
            "pjrc" in mfr or
            (port.vid == 0x16C0 and port.pid == 0x0483)  # Teensy USB VID:PID
        )
        
        marker = " ← TEENSY" if is_teensy else ""
        print(f"  {port.device}{marker}")
        print(f"    Description: {port.description}")
        print(f"    Manufacturer: {port.manufacturer}")
        print(f"    VID:PID: {hex(port.vid)}:{hex(port.pid)}" if port.vid else "    VID:PID: N/A")
        print()
        
        if is_teensy and teensy_port is None:
            teensy_port = port.device

    print("─" * 50)
    if teensy_port:
        print(f"✓ Teensy detected: {teensy_port}")
    else:
        print("❌ No Teensy detected - hardware mode will fail!")
        print("   Connect Teensy and re-run this cell.")
else:
    print("ℹ️ Digital twin mode - skipping serial port detection")

Scanning serial ports...

  /dev/ttyS31
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS30
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS29
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS28
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS27
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS26
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS25
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS24
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS23
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS22
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS21
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS20
    Description: n/a
    Manufacturer: None
    VID:PID: N/A

  /dev/ttyS19
    Description: n/a
    Manufacturer: None
    

---
## 2. Setup

Import the motor architecture from telescope-mcp.

In [3]:
# ============================================
# SETUP - Import motor architecture
# Run this after kernel restart to pick up code changes
# ============================================

# Force reimport of modules (useful during development)
# Order matters! Reload base types first, then implementations
import importlib
import telescope_mcp.drivers.motors.types
import telescope_mcp.drivers.motors.twin
import telescope_mcp.drivers.motors.serial_controller  # Hardware driver
import telescope_mcp.drivers.motors  # Also reload the __init__.py

importlib.reload(telescope_mcp.drivers.motors.types)
importlib.reload(telescope_mcp.drivers.motors.twin)
importlib.reload(telescope_mcp.drivers.motors.serial_controller)
importlib.reload(telescope_mcp.drivers.motors)

from telescope_mcp.drivers.motors import (
    DigitalTwinMotorDriver,
    DigitalTwinMotorConfig,
    DigitalTwinMotorInstance,
    SerialMotorDriver,  # Real hardware driver
    MotorStatus,
    MotorInfo,
    MotorType,
)
from telescope_mcp.devices import Motor, MotorConfig
import asyncio

print("✓ Motor architecture imported")

✓ Motor architecture imported


In [4]:
# ============================================
# CREATE MOTOR DRIVER
# Uses hardware or digital twin based on USE_HARDWARE setting
# ============================================

if USE_HARDWARE:
    # Real hardware - connect to Teensy motor controller
    if not teensy_port:
        raise RuntimeError(
            "No Teensy detected! Connect the motor controller and re-run the port detection cell."
        )
    
    serial_port = teensy_port
    driver = SerialMotorDriver(baudrate=MOTOR_BAUD_RATE)
    print(f"✓ SerialMotorDriver created")
    print(f"  Serial port: {serial_port}")
    print(f"  Baud rate: {MOTOR_BAUD_RATE}")
else:
    # Digital twin - simulated motors
    serial_port = None  # Not used in digital twin mode
    config = DigitalTwinMotorConfig(
        # Position limits (matching real hardware)
        altitude_min_steps=0,       # Zenith (straight up)
        altitude_max_steps=140000,  # Horizon
        azimuth_min_steps=-110000,  # CCW limit
        azimuth_max_steps=110000,   # CW limit
        
        # Home positions
        altitude_home_steps=0,      # Home at zenith
        azimuth_home_steps=0,       # Home at center
        
        # Timing simulation
        slew_speed_steps_per_sec=5000.0,
        acceleration_time_sec=0.5,
        simulate_timing=True,
    )
    driver = DigitalTwinMotorDriver(config)
    print(f"✓ DigitalTwinMotorDriver created")
    print(f"  Config: {config}")

✓ SerialMotorDriver created
  Serial port: /dev/ttyACM2
  Baud rate: 115200


In [5]:
# ============================================
# OPEN MOTOR CONTROLLER
# ============================================

# Open motor controller (pass port for hardware mode)
if USE_HARDWARE:
    motor = driver.open(serial_port)
else:
    motor = driver.open()

info = motor.get_info()
print(f"✓ Motor controller opened: {info['name']}")

# Show connection settings
print(f"\n{'═' * 50}")
print("CONNECTION")
print(f"{'═' * 50}")
if USE_HARDWARE:
    print(f"  Port:      {serial_port}")
    print(f"  Baud rate: {MOTOR_BAUD_RATE}")
else:
    print(f"  Mode:      Digital Twin (simulated)")

# Show motor configuration
print(f"\n{'═' * 50}")
print("MOTOR CONFIGURATION")
print(f"{'═' * 50}")

# Import motor configs for display
from telescope_mcp.drivers.motors.serial_controller import MOTOR_CONFIGS

for motor_type in [MotorType.ALTITUDE, MotorType.AZIMUTH]:
    cfg = MOTOR_CONFIGS[motor_type]
    print(f"\n  {motor_type.value.upper()}:")
    print(f"    Range:      {cfg.min_steps:,} → {cfg.max_steps:,} steps")
    print(f"    Home:       {cfg.home_position:,} steps")
    print(f"    Steps/deg:  {cfg.steps_per_degree:,.1f}")

# Query firmware help (hardware only)
if USE_HARDWARE:
    print(f"\n{'═' * 50}")
    print("TEENSY FIRMWARE COMMANDS")
    print(f"{'═' * 50}")
    try:
        help_text = motor.get_help()
        # Indent each line for cleaner display
        for line in help_text.strip().split('\n'):
            if line.strip():
                print(f"  {line.strip()}")
    except Exception as e:
        print(f"  (Could not query firmware: {e})")

# Get initial positions
print(f"\n{'═' * 50}")
print("CURRENT POSITIONS")
print(f"{'═' * 50}")
alt_pos = motor.get_status(MotorType.ALTITUDE)
az_pos = motor.get_status(MotorType.AZIMUTH)
print(f"  ALT: {alt_pos.position_steps:,} steps")
print(f"  AZ:  {az_pos.position_steps:,} steps")

2026-01-04 10:12:15,002 - telescope_mcp.drivers.motors.serial_controller - INFO - Motor controller connected | port=/dev/ttyACM2


✓ Motor controller opened: Serial Motor Controller (/dev/ttyACM2)

══════════════════════════════════════════════════
CONNECTION
══════════════════════════════════════════════════
  Port:      /dev/ttyACM2
  Baud rate: 115200

══════════════════════════════════════════════════
MOTOR CONFIGURATION
══════════════════════════════════════════════════

  ALTITUDE:
    Range:      0 → 140,000 steps
    Home:       0 steps
    Steps/deg:  1,555.6

  AZIMUTH:
    Range:      -110,000 → 110,000 steps
    Home:       0 steps
    Steps/deg:  814.8

══════════════════════════════════════════════════
TEENSY FIRMWARE COMMANDS
══════════════════════════════════════════════════
  stepper_amis command set
  Ground Teensy pin 15 to disable joysticks for system setup
  f = float, n = integer, both end with space
  Ad	Select current axis to d (0 to 2)
  B	Read joyMax for selected axis
  C	Get motion status
  Dd	Set number of microsteps per step for selected axis
  Ed	EEPROM (0 = load, 1 = save, 2 = scramb

---
## 2.1 StepperAmis Raw Diagnostics

**Direct access to hardware settings via stepperamis library.**

Use this section to:
- Read ALL motor controller parameters from the Teensy
- Check velocity, acceleration, current settings
- Identify misconfiguration causing movement issues

In [6]:
# ============================================
# STEPPER_AMIS RAW CONNECTION
# Direct connection bypassing telescope-mcp for diagnostics
# ============================================
import sys
sys.path.insert(0, '/home/mark/src/at_stepper_amis')

from stepper_amis import Stepper_amis, getAllHwids
from pprint import pprint

# Show available serial devices
print("Available serial devices:")
pprint(getAllHwids())

if USE_HARDWARE and teensy_port:
    # Close telescope-mcp connection first to avoid port conflict
    try:
        driver.close()
        print("\n✓ Closed telescope-mcp driver to release serial port")
    except:
        pass
    
    # Use known hwid if configured, otherwise search for it
    if TEENSY_HWID:
        teensy_hwid = TEENSY_HWID
        print(f"\nUsing configured Teensy hwid: {teensy_hwid}")
    else:
        # Find hwid for the Teensy port (fallback)
        from serial.tools.list_ports import comports
        teensy_hwid = None
        for cp in comports():
            if str(cp).split()[0] == teensy_port:
                teensy_hwid = cp.hwid
                break
        
        if teensy_hwid:
            print(f"\nTeensy hwid (detected): {teensy_hwid}")
            print(f"💡 TIP: Add this to TEENSY_HWID in config cell for faster connection")
        else:
            print(f"❌ Could not find hwid for {teensy_port}")
    
    if teensy_hwid:
        # Connect using hwid (Stepper_amis API)
        sa = Stepper_amis(teensy_hwid)
        print(f"✓ Stepper_amis connected via {teensy_port}")
    else:
        sa = None
else:
    print("❌ Hardware mode not enabled or no Teensy port detected")
    sa = None

2026-01-04 10:12:15,188 - telescope_mcp.drivers.motors.serial_controller - INFO - Motor controller closed | port=/dev/ttyACM2


Available serial devices:
{'/dev/ttyACM0': {'hwid': 'USB VID:PID=2341:805A SER=D6887F2418ABF612 '
                          'LOCATION=2-2:1.0'},
 '/dev/ttyACM2': {'hwid': 'USB VID:PID=16C0:0483 SER=4516910 '
                          'LOCATION=2-3.4:1.0'},
 '/dev/ttyS0': {'hwid': 'n/a'},
 '/dev/ttyS1': {'hwid': 'n/a'},
 '/dev/ttyS10': {'hwid': 'n/a'},
 '/dev/ttyS11': {'hwid': 'n/a'},
 '/dev/ttyS12': {'hwid': 'n/a'},
 '/dev/ttyS13': {'hwid': 'n/a'},
 '/dev/ttyS14': {'hwid': 'n/a'},
 '/dev/ttyS15': {'hwid': 'n/a'},
 '/dev/ttyS16': {'hwid': 'n/a'},
 '/dev/ttyS17': {'hwid': 'n/a'},
 '/dev/ttyS18': {'hwid': 'n/a'},
 '/dev/ttyS19': {'hwid': 'n/a'},
 '/dev/ttyS2': {'hwid': 'n/a'},
 '/dev/ttyS20': {'hwid': 'n/a'},
 '/dev/ttyS21': {'hwid': 'n/a'},
 '/dev/ttyS22': {'hwid': 'n/a'},
 '/dev/ttyS23': {'hwid': 'n/a'},
 '/dev/ttyS24': {'hwid': 'n/a'},
 '/dev/ttyS25': {'hwid': 'n/a'},
 '/dev/ttyS26': {'hwid': 'n/a'},
 '/dev/ttyS27': {'hwid': 'n/a'},
 '/dev/ttyS28': {'hwid': 'n/a'},
 '/dev/ttyS29': {'hw

In [7]:
# ============================================
# GET ALL SETTINGS FOR ALL AXES
# Retrieves velocity, acceleration, current, position for each axis
# ============================================

print("=" * 60)
print("STEPPER AMIS FULL HARDWARE SETTINGS")
print("=" * 60)

if sa is None:
    print("❌ No connection - run the connection cell first")
else:
    for axis in range(3):
        # status() = major items (G command), status2() = minor items (g command)
        major = sa.status(axis)
        minor = sa.status2(axis)
        
        print(f"\n{'─' * 60}")
        print(f"AXIS {axis}")
        print(f"{'─' * 60}")
        
        # Key motor parameters from status (G command)
        print(f"  Position (x):     {major.get('x', 'N/A'):>10} microsteps")
        print(f"  Velocity (speed): {major.get('speed', 'N/A'):>10} μsteps/sec")
        print(f"  Accel (accel):    {major.get('accel', 'N/A'):>10} seconds")
        print(f"  Microsteps (us):  {major.get('us', 'N/A'):>10}")
        
        # Current settings
        print(f"  Current (iRun):   {major.get('iRun', 'N/A'):>10} mA")
        print(f"  Current (iHold):  {major.get('iHold', 'N/A'):>10} mA")
        
        # Limit sensor
        print(f"  Limit Enabled:    {major.get('lim', 'N/A'):>10}")
        print(f"  Limit State:      {major.get('limState', 'N/A'):>10}")
        
        # Polarity
        print(f"  Direction Pol:    {major.get('dirPol', 'N/A'):>10}")
        
        # From status2 (g command)
        print(f"  Joystick Enable:  {minor.get('jEn', 'N/A'):>10}")
        print(f"  Joystick Zero:    {minor.get('jz', 'N/A'):>10}")

    # Show raw dicts for both motor axes
    for axis in [0, 1]:
        axis_name = "ALTITUDE" if axis == 0 else "AZIMUTH"
        print(f"\n{'=' * 60}")
        print(f"RAW STATUS DICTS - AXIS {axis} ({axis_name})")
        print("=" * 60)
        print("\nstatus() [G command]:")
        pprint(sa.status(axis))
        print("\nstatus2() [g command]:")
        pprint(sa.status2(axis))

STEPPER AMIS FULL HARDWARE SETTINGS

────────────────────────────────────────────────────────────
AXIS 0
────────────────────────────────────────────────────────────
  Position (x):              0 microsteps
  Velocity (speed):     4000.0 μsteps/sec
  Accel (accel):           0.2 seconds
  Microsteps (us):         N/A
  Current (iRun):          N/A mA
  Current (iHold):         N/A mA
  Limit Enabled:           N/A
  Limit State:             N/A
  Direction Pol:           N/A
  Joystick Enable:         N/A
  Joystick Zero:           N/A

────────────────────────────────────────────────────────────
AXIS 1
────────────────────────────────────────────────────────────
  Position (x):              0 microsteps
  Velocity (speed):     4000.0 μsteps/sec
  Accel (accel):           0.2 seconds
  Microsteps (us):         N/A
  Current (iRun):          N/A mA
  Current (iHold):         N/A mA
  Limit Enabled:           N/A
  Limit State:             N/A
  Direction Pol:           N/A
  Joystick E

In [8]:
# ============================================
# FIX MOTOR PARAMETERS (If needed)
# Uses Stepper_amis API: setSpeed, iMotor, iHold, setATime, usMotor
# ============================================

# ═══════════════════════════════════════════════════════════════
# AXIS MAPPING (confirmed 2026-01-03)
# ═══════════════════════════════════════════════════════════════
#   Axis 0 = ALTITUDE (ALT)
#   Axis 1 = AZIMUTH (AZ)
#   Axis 2 = (unused)
# ═══════════════════════════════════════════════════════════════

# ═══════════════════════════════════════════════════════════════
# MOTOR SPECIFICATIONS: 17HS24-2104S (NEMA 17)
# ═══════════════════════════════════════════════════════════════
# Manufacturer Part Number: 17HS24-2104S
# Motor Type:               Bipolar Stepper
# Step Angle:               1.8° (200 steps/revolution)
# Holding Torque:           65 Ncm (92 oz.in)
# Rated Current/phase:      2.1A
# Phase Resistance:         1.6Ω
# Inductance:               3mH ±20% (1KHz)
# Insulation Class:         B 130°C [266°F]
#
# MICROSTEP CALCULATION:
#   At 128 microsteps: 200 × 128 = 25,600 μsteps/revolution
# ═══════════════════════════════════════════════════════════════

# Select axis to configure
AXIS_TO_FIX = 0  # ALTITUDE motor

# Recommended settings based on 17HS24-2104S specs
# ─────────────────────────────────────────────────────────────
VELOCITY = 4000           # μsteps/sec - reasonable speed
ACCEL_TIME = 0.2          # seconds to reach velocity  
CURRENT_MOVING = 2100     # mA (100% of 2.1A rated - good torque, less heat)
CURRENT_HOLDING = 400     # mA (~25% of moving - prevents overheating)
MICROSTEPS = 128          # microstepping divisor (8, 16, 32, 64, 128)
DIR_POLARITY = 1          # 1 or -1 for direction
# ─────────────────────────────────────────────────────────────

# APPLY SETTINGS
print(f"Applying settings to Axis {AXIS_TO_FIX} (ALTITUDE)...")
print(f"  Motor: 17HS24-2104S (2.1A rated)")
print(f"  Current: {CURRENT_MOVING}mA moving / {CURRENT_HOLDING}mA hold (75% rated)")
sa.setSpeed(AXIS_TO_FIX, VELOCITY)
sa.setATime(AXIS_TO_FIX, ACCEL_TIME)
sa.iMotor(AXIS_TO_FIX, CURRENT_MOVING)
sa.iHold(AXIS_TO_FIX, CURRENT_HOLDING)
sa.usMotor(AXIS_TO_FIX, MICROSTEPS)
sa.setDirPol(AXIS_TO_FIX, DIR_POLARITY)
print("✓ Settings applied (RAM only, not saved to EEPROM)")

# Verify
print("\nVerifying new settings:")
pprint(sa.status(AXIS_TO_FIX))

Applying settings to Axis 0 (ALTITUDE)...
  Motor: 17HS24-2104S (2.1A rated)
  Current: 2100mA moving / 400mA hold (75% rated)
✓ Settings applied (RAM only, not saved to EEPROM)

Verifying new settings:
{'accel': 0.2,
 'axis': 0,
 'dirPolarity': 1,
 'joyEnable': 0,
 'limit': 1,
 'limitEnable': 0,
 'limitPolarity': 0,
 'speed': 4000.0,
 'units': 'microns',
 'units_step': 1.5,
 'usteps': 128,
 'x': 0}


In [37]:
# ============================================
# FIX AZIMUTH MOTOR PARAMETERS (If needed)
# Uses Stepper_amis API: setSpeed, iMotor, iHold, setATime, usMotor
# ============================================

# ═══════════════════════════════════════════════════════════════
# MOTOR SPECIFICATIONS: 23HS41-1804S (NEMA 23)
# ═══════════════════════════════════════════════════════════════
# Manufacturer Part Number: 23HS41-1804S
# Motor Type:               Bipolar Stepper
# Step Angle:               1.8° (200 steps/revolution)
# Holding Torque:           ~1.26 Nm (179 oz.in)
# Rated Current/phase:      1.8A
# Phase Resistance:         1.5Ω
# Inductance:               3.6mH ±20%
#
# NOTE: Larger NEMA 23 motor for azimuth axis (horizontal rotation)
#       No gravity load = lower holding current needed
# ═══════════════════════════════════════════════════════════════

# Select axis to configure
AXIS_TO_FIX = 1  # AZIMUTH motor

# Recommended settings based on 23HS41-1804S specs
# ─────────────────────────────────────────────────────────────
VELOCITY = 4000           # μsteps/sec - slower for max torque headroom
ACCEL_TIME = 0.3         # seconds to reach velocity (gentle ramp = less peak torque demand)
CURRENT_MOVING = 1800     # mA (100% of 1.8A rated - full torque)
CURRENT_HOLDING = 100     # mA - horizontal axis, minimal gravity load
MICROSTEPS = 128        # microstepping divisor (lower = more torque)
DIR_POLARITY = 1          # 1 or -1 for direction
# ─────────────────────────────────────────────────────────────

# APPLY SETTINGS
print(f"Applying settings to Axis {AXIS_TO_FIX} (AZIMUTH)...")
print(f"  Motor: 23HS41-1804S (1.8A rated, NEMA 23)")
print(f"  Velocity: {VELOCITY} μsteps/sec (slower for more torque)")
print(f"  Current: {CURRENT_MOVING}mA moving / {CURRENT_HOLDING}mA hold")
sa.setSpeed(AXIS_TO_FIX, VELOCITY)
sa.setATime(AXIS_TO_FIX, ACCEL_TIME)
sa.iMotor(AXIS_TO_FIX, CURRENT_MOVING)
sa.iHold(AXIS_TO_FIX, CURRENT_HOLDING)
sa.usMotor(AXIS_TO_FIX, MICROSTEPS)
sa.setDirPol(AXIS_TO_FIX, DIR_POLARITY)
print("✓ Settings applied (RAM only, not saved to EEPROM)")

# Verify
print("\nVerifying new settings:")
pprint(sa.status(AXIS_TO_FIX))

Applying settings to Axis 1 (AZIMUTH)...
  Motor: 23HS41-1804S (1.8A rated, NEMA 23)
  Velocity: 4000 μsteps/sec (slower for more torque)
  Current: 1800mA moving / 100mA hold


SerialException: write failed: [Errno 5] Input/output error

In [36]:
# ============================================
# TEST MOVE WITH STEPPER_AMIS DIRECTLY
# Bypasses telescope-mcp to test raw motor movement
# ============================================

TEST_AXIS = 1
#TEST_MICROSTEPS = 64000 # ~8 full steps at 128 microsteps/step
TEST_MICROSTEPS = -64000 # ~8 full steps at 128 microsteps/step

if sa is None:
    print("❌ No connection - run the connection cell first")
else:
    print(f"Testing Axis {TEST_AXIS} with {TEST_MICROSTEPS} microsteps...")

    # Get position before
    status_before = sa.status(TEST_AXIS)
    pos_before = status_before.get('x', 0)
    print(f"  Position before: {pos_before}")

    # Get current settings for this move
    vel = status_before.get('speed', 0)
    accel = status_before.get('accel', 0)
    current = status_before.get('iRun', 0)
    print(f"  Velocity: {vel} μsteps/sec")
    print(f"  Acceleration: {accel} sec")
    print(f"  Current: {current} mA")

    # Do the move (move2 is relative move)
    print(f"\n  Moving +{TEST_MICROSTEPS} microsteps...")
    sa.move2(TEST_AXIS, TEST_MICROSTEPS)

    # Check new position
    status_after = sa.status(TEST_AXIS)
    pos_after = status_after.get('x', 0)
    delta = pos_after - pos_before

    print(f"\n  Position after: {pos_after}")
    print(f"  Actual delta: {delta} microsteps")

    if abs(delta) >= abs(TEST_MICROSTEPS) * 0.9:
        print(f"  ✓ MOVEMENT SUCCESSFUL")
    elif delta != 0:
        print(f"  ⚠️ PARTIAL MOVEMENT (expected {TEST_MICROSTEPS}, got {delta})")
    else:
        print(f"  ❌ NO MOVEMENT DETECTED")
        print(f"     Check: velocity, current, mechanical binding")

Testing Axis 1 with -64000 microsteps...
  Position before: 192011
  Velocity: 4000.0 μsteps/sec
  Acceleration: 0.2 sec
  Current: 0 mA

  Moving +-64000 microsteps...

  Position after: 128012
  Actual delta: -63999 microsteps
  ✓ MOVEMENT SUCCESSFUL


In [ ]:
# ============================================
# SAVE ALL SETTINGS TO EEPROM
# Settings will persist across power cycles
# ============================================

print("Saving motor settings to EEPROM...")
print("  ⚠️ Settings will persist across power cycles\n")

# Save ALTITUDE (Axis 0)
print("Saving ALTITUDE (Axis 0)...")
sa.storeEE(0, 1)
print("  ✓ ALTITUDE saved to EEPROM")

# Save AZIMUTH (Axis 1)
print("Saving AZIMUTH (Axis 1)...")
sa.storeEE(1, 1)
print("  ✓ AZIMUTH saved to EEPROM")

print("\n" + "=" * 50)
print("✓ ALL SETTINGS SAVED TO EEPROM")
print("=" * 50)

In [ ]:
# ============================================
# CLOSE STEPPER_AMIS CONNECTION
# Run before going back to telescope-mcp driver
# ============================================

try:
    sa.close()
    print("✓ Stepper_amis serial connection closed")
    print("  You can now re-run the telescope-mcp driver cells")
except Exception as e:
    print(f"Connection close error: {e}")
    print("  You can now re-run the telescope-mcp driver cells")
except:
    print("Connection already closed or not opened")